In [60]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

# Selecionando websites


Criando um dicionário com o site e uma lista de páginas.

In [35]:
url_dict = {
    'capricho':
    [
        'https://capricho.abril.com.br/beleza/fenty-beauty-lanca-iluminador-que-promete-favorecer-todos-os-tons-de-pele/'
    ]
    ,'dia de beaute':
    [
        'https://diadebeaute.com/2024/03/21/lollapalooza-dicas-para-fazer-sua-make-durar-no-festival/'
        ,'https://diadebeaute.com/2024/03/15/bases-queridinhas-dos-maquiadores-das-celebridades/'
    ]
}

# Converte o dicionário em uma lista de dicionários
data = [{'site': key, 'url': url} for key, urls in url_dict.items() for url in urls]

# Transforma em um DataFrame
url_df = pd.DataFrame(data)
url_df

,site,url
0,capricho,https://capricho.abril.com.br/beleza/fenty-bea...
1,dia de beaute,https://diadebeaute.com/2024/03/21/lollapalooz...
2,dia de beaute,https://diadebeaute.com/2024/03/15/bases-queri...


In [7]:
# Puxando stopwords do Natural Language ToolKit (NLTK)
nltk.download('stopwords')
stopAll = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /home/vinifm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
def get_page(url):
    response = requests.get(url)
    content = response.content
    soup = bs(content, 'html.parser')
    return soup.get_text()

def clean_content(pages: list) -> list:
    cleaned_content = []
    for content in pages:
        content = re.sub(r'\s+', ' ', content)

        # remove pontuação
        punctuation = string.punctuation + "-–"
        content = ''.join(char for char in content if char not in punctuation)
        cleaned_content.append(content)

    return cleaned_content

# Adiciona uma coluna no dataframe com o conteúdo da página
def get_pages_from_dict(url_df: pd.DataFrame) -> pd.DataFrame:
    results = map(get_page, list(url_df['url']))
    cleaned_content = clean_content(list(results))
    url_df['content'] = cleaned_content

    words = []
    for page in url_df['content']:
        words.append([word.lower() for word in page.split() if word not in stopAll])
    url_df['words'] = words

    return url_df

In [93]:
df = get_pages_from_dict(url_df)
df

,site,url,content,words
0,capricho,https://capricho.abril.com.br/beleza/fenty-bea...,Fenty Beauty lança iluminador que promete fav...,"[fenty, beauty, lança, iluminador, promete, fa..."
1,dia de beaute,https://diadebeaute.com/2024/03/21/lollapalooz...,Lollapalooza dicas para fazer sua make durar ...,"[lollapalooza, dicas, fazer, make, durar, fest..."
2,dia de beaute,https://diadebeaute.com/2024/03/15/bases-queri...,Bases queridinhas dos maquiadores das celebri...,"[bases, queridinhas, maquiadores, celebridades..."


Como queremos contabilizar palavras a nível de site e não página, juntamos as listas de palavras de cada página por site.


In [94]:
grouped_df = df.groupby('site')['words'].agg(lambda x: sum(x, [])).reset_index()
grouped_df

,site,words
0,capricho,"[fenty, beauty, lança, iluminador, promete, fa..."
1,dia de beaute,"[lollapalooza, dicas, fazer, make, durar, fest..."


# Configurando o PySpark

In [95]:
from pyspark.sql import SparkSession

# Exit em processos pré-existentes
if 'sc' in locals():
    sc.stop()

spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
sc = spark.sparkContext

### Criando um RDD
Um RDD (Resilient Distributed Dataset) no PySpark é a estrutura de dados fundamental do Apache Spark, representando uma coleção distribuída de elementos que podem ser processados em paralelo. Ele é projetado para ser imutável e tolerante a falhas, permitindo operações de transformação (como map e filter) e ações (como count e collect) de forma eficiente e distribuída. Os RDDs podem ser criados a partir de dados presentes no sistema de arquivos local, HDFS, ou outras fontes de dados, e são fundamentais para o processamento e análise de grandes volumes de dados em ambientes distribuíd

In [96]:
df_word_count = pd.DataFrame()
site_list = []
word_count_list = []
for index, row in grouped_df.iterrows():
    rdd = sc.parallelize(row['words'])
    counts = rdd.map(lambda word:(word, 1)).reduceByKey(lambda a, b: a + b)
    count_list = sorted(counts.collect(), key=lambda x: x[1], reverse=True)

    # filtra somente palavras que ocorrem mais de duas vezes
    filtered_count = [item for item in count_list if item[1] > 2]
    word_count_list.append(filtered_count)
    site_list.append(row['site'])

df_word_count['site'] = site_list
df_word_count['word_count'] = word_count_list

df_word_count

,site,word_count
0,capricho,"[(fenty, 15), (beauty, 14), (a, 12), (publicid..."
1,dia de beaute,"[(pele, 19), (aqui, 18), (base, 16), (aquibase..."


In [97]:
df_word_count['word_count'][1]

[('pele', 19),
 ('aqui', 18),
 ('base', 16),
 ('aquibase', 13),
 ('make', 13),
 ('the', 12),
 ('maquiagem', 12),
 ('to', 10),
 ('beauté', 10),
 ('beleza', 8),
 ('bem', 8),
 ('opens', 8),
 ('in', 8),
 ('new', 8),
 ('o', 8),
 ('on', 8),
 ('share', 8),
 ('bases', 8),
 ('batom', 8),
 ('cookies', 8),
 ('terracotta', 7),
 ('dior', 7),
 ('vic', 7),
 ('forever', 7),
 ('natural', 7),
 ('sobre', 7),
 ('backstage', 7),
 ('…', 6),
 ('armazenamento', 6),
 ('acesso', 6),
 ('guerlain', 6),
 ('le', 6),
 ('leia', 6),
 ('funcional', 6),
 ('preferences', 6),
 ('beauty', 6),
 ('mais', 6),
 ('preferências', 6),
 ('dia', 6),
 ('técnico', 6),
 ('marketing', 6),
 ('minutos', 6),
 ('tudo', 6),
 ('or', 6),
 ('manage', 6),
 ('cobertura', 6),
 ('produtos', 6),
 ('skin', 6),
 ('windowclick', 6),
 ('site', 6),
 ('teint', 5),
 ('necessário', 5),
 ('sempre', 5),
 ('lauder', 5),
 ('cor', 4),
 ('todosviagemalém', 4),
 ('delineador', 4),
 ('storage', 4),
 ('estatísticas', 4),
 ('dicas', 4),
 ('beautéver', 4),
 ('vários'